In [114]:
#Import all the needed libraries
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import seaborn as sns
from mizani.formatters import percent_format
import os
from plotnine import *
import numpy as np
import sys
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from stargazer import stargazer
from statsmodels.tools.eval_measures import mse,rmse

In [115]:
#Current working directory
path = os.getcwd()
path

'C:\\Users\\Anna\\predictions_hw1'

In [116]:
#Give a new path
os.chdir('C:\\Users\\Anna\\predictions_hw1')


In [117]:
dirname = os.getcwd()
data_in = dirname + "da_data_repo/cps-earnings/clean/"
data_out = dirname + "da_case_studies/ch09-gender-age-earnings/"

In [118]:
#Import the data
data = pd.read_csv('morg-2014-emp.csv')
data.head()#take a look at the data

,Unnamed: 0,hhid,intmonth,stfips,weight,earnwke,uhours,grade92,race,ethnic,...,ownchild,chldpres,prcitshp,state,ind02,occ2012,class,unionmme,unioncov,lfsr94
0,3,2600310997690,January,AL,3151.6801,1692.00,40,43,1,NaN,...,0,0,"Native, Born In US",63,Employment services (5613),630,"Private, For Profit",No,No,Employed-At Work
1,5,75680310997590,January,AL,3457.1138,450.00,40,41,2,NaN,...,2,6,"Native, Born In US",63,Outpatient care centers (6214),5400,"Private, For Profit",No,No,Employed-Absent
2,6,75680310997590,January,AL,3936.9110,1090.00,60,41,2,NaN,...,2,6,"Native, Born In US",63,Motor vehicles and motor vehicle equipment man...,8140,"Private, For Profit",No,No,Employed-At Work
3,10,179140131100930,January,AL,3288.3640,769.23,40,40,1,NaN,...,2,4,"Native, Born In US",63,"**Publishing, except newspapers and software (...",8255,"Private, For Profit",Yes,NaN,Employed-At Work
4,11,179140131100930,January,AL,3422.8500,826.92,40,43,1,NaN,...,2,4,"Native, Born In US",63,"Banking and related activities (521, 52211,52219)",5940,"Private, For Profit",No,No,Employed-At Work


In [119]:
#Filter the data in a way that I only have doctor's occupations. Excluded: pharmacists; nurses; variables including "all other";
# veterinarians, opticions, technologists and technicians
df = data.loc[data.occ2012.isin([3000, 3010, 3030, 3040, 3060, 3140, 3150, 3160, 3200, 3210, 3220])]

df.head() #check how this new data set looks like
len(df.columns) #check whether it has same number of columns

23

In [123]:
# Create new variables
df["female"] = (df.sex == 2).astype(int) #transform sex variable in easier to use and read dummy variable
df["wage"] = df["earnwke"] / df["uhours"] #variable for hourly wage
df["lnwage"] = np.log(df["wage"]) # I need a log of hours wage
df["agesq"] = np.power(df["age"], 2) # variable for age squared since I want to test the quadratic relationship between ln hourly wage and wage
df["fem_age"] = df['female'] * df['age'] # including interaction to test whether gender differential varies by age
 
df['citizen'] = df.prcitshp.apply(lambda x: 1 if x != "Foreign Born, Not a US Citizen" else 0)#dummy to account that some doctors are officialy citizens (1 = citizen)
df['spouce'] = df.marital.apply(lambda x: 1 if x <= 2 else 0) # create dummy 1 - if spouce is married and present, 0 otherwise
df['atwork'] = df.lfsr94.apply(lambda x: 1 if x == "Employed-At Work" else 0) # 1 - emp at work, 0 - emp, absent
df['private'] = df["class"].apply(lambda x: 1 if x == 'Private, For Profit' or x == 'Private, Nonprofit' else 0) 


count    1694.000000
mean        0.879575
std         0.325554
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: private, dtype: float64

In [124]:
#I suspect that those who are employed but not at work might be significantly different from those who are at work right now
# Moreover, I don't know for what reasons they are not at work (pregnancy might be one, but might be smth else)
# So, I drop all the employed-absent variables from data set
df = df[df.atwork != 0] #drop those absent at work
df.atwork.describe() #check whether it is what I wanted now

count    1629.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
Name: atwork, dtype: float64

In [125]:
#Sample data set only to variables of interest
list(df.columns) #look at all the names of columns

['Unnamed: 0',
 'hhid',
 'intmonth',
 'stfips',
 'weight',
 'earnwke',
 'uhours',
 'grade92',
 'race',
 'ethnic',
 'age',
 'sex',
 'marital',
 'ownchild',
 'chldpres',
 'prcitshp',
 'state',
 'ind02',
 'occ2012',
 'class',
 'unionmme',
 'unioncov',
 'lfsr94',
 'female',
 'wage',
 'lnwage',
 'agesq',
 'fem_age',
 'citizen',
 'spouce',
 'atwork',
 'private']

In [126]:
# Select columns which are of interest
df_final = df[['hhid', 'earnwke','uhours','grade92', 'age','agesq', 'ownchild', 'occ2012', 'private','female','lnwage','fem_age','citizen',
              'spouce', 'atwork' ]]